In [1]:
import sys
sys.path.append('/home/kai/DAST/network')
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import torch.utils.data.dataloader as Data
import os
import time
from torch.autograd import Variable
from DAST_utils import *
from DAST_Network import *
from torch.utils.data import TensorDataset,DataLoader
import sys
from tqdm import tqdm
import math

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
#Myscore function
def myScore(target, pred):
    print('target', target.shape)
    print('pred', pred.shape)
    Er = 100 * ((target - pred) / target)
    print(Er.shape)
    A = math.exp(-np.log(0.5) * (Er / 5)) if Er <=0 else math.exp(np.log(0.5) * (Er / 20))
    return A

In [4]:
def RMSE(target, pred):
    square_error = (target - pred) ** 2
    mse =  (torch.sum(square_error)) / len(target)
    rmse = mse ** 0.5
    return rmse

In [5]:
def MAE(target, pred):
    absolute_error = np.abs(target - pred)
    return torch.sum(absolute_error) / len(target)

In [6]:
DATA_PATH = '../../../data/10FEMTO/processed_data/'
FOLDER = 'Bearing1_1'

In [7]:
prediction_loss = []
# Load preprocessed data
X_train = np.load(f'{DATA_PATH}/{FOLDER}/{FOLDER}_train_X.npy')
Y_train = np.load(f'{DATA_PATH}/{FOLDER}/{FOLDER}_train_Y.npy')

FOLDER = 'Bearing1_3'
# Y_train = Y_train.transpose()
X_test = np.load(f'{DATA_PATH}/{FOLDER}/{FOLDER}_test_X.npy')
Y_test = np.load(f'{DATA_PATH}/{FOLDER}/{FOLDER}_test_Y.npy')

In [8]:
Y_train = np.reshape(Y_train, (len(Y_train), -1))
Y_test = np.reshape(Y_test, (len(Y_test), -1))
print(Y_train.shape)
print(Y_test.shape)

(2763, 1)
(1762, 1)


In [9]:
X_train = Variable(torch.Tensor(X_train).float())
Y_train = Variable(torch.Tensor(Y_train).float())
X_test = Variable(torch.Tensor(X_test).float())
Y_test = Variable(torch.Tensor(Y_test).float())

In [10]:
#Hyperparameters
batch_size = 256
dim_val = 20
dim_attn = 20
dim_val_t = 20
dim_attn_t = 20
dim_val_s = 20
dim_attn_s = 20
n_heads = 4
n_decoder_layers = 1
n_encoder_layers = 2
max_rul = 125
lr = 1e-5
epochs = 50
time_step = 40
dec_seq_len = 4
output_sequence_length = 1
input_size = 20

In [11]:
train_dataset = TensorDataset(X_train,Y_train)
train_loader = Data.DataLoader(dataset=train_dataset,batch_size = batch_size,shuffle=True)
test_dataset = TensorDataset(X_test,Y_test)
test_loader = Data.DataLoader(dataset=test_dataset,batch_size = batch_size,shuffle=False)

In [12]:
# Initialize model parameters
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# debug = True
debug = False
model = DAST(dim_val_s,dim_attn_s,dim_val_t,dim_attn_t,dim_val, dim_attn,time_step,input_size,dec_seq_len,output_sequence_length, n_decoder_layers, n_encoder_layers, n_heads, debug)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

In [13]:
loss_list = []
train_loss_list = []
test_loss_list = []
train_time = []
test_time = []

In [14]:
for epoch in range(epochs):
    #training
    model.train()
    loop = tqdm(train_loader, leave=True)
    for i,(X, Y) in enumerate(loop):
        batch_X = X.to(device)
        batch_Y = Y.to(device)
        out = model(batch_X)
        loss = criterion(out, batch_Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    loss_eopch = np.mean(np.array(loss_list))
    train_loss_list.append(loss_eopch)
    print('epoch = ',epoch,
            'train_loss = ',loss_eopch.item())

    #testing
    model.eval()
    prediction_list = []
    for j ,(batch_x,batch_y) in enumerate(test_loader):
        batch_X = batch_x.to(device)
        batch_Y = batch_y.to(device)
        prediction = model(batch_X)
        prediction[prediction<0] = 0
        prediction_list.append(prediction)

    out_batch_pre = torch.cat(prediction_list).detach().cpu().numpy()
    prediction_tensor = torch.from_numpy(out_batch_pre)
    rmse_loss = RMSE(Y_test, prediction_tensor, )
    mae_loss = MAE(Y_test, prediction_tensor, )
    print('rmse_loss = ', rmse_loss.item(),
            'mae_loss = ', mae_loss.item())

100%|██████████| 11/11 [00:00<00:00, 20.33it/s]


epoch =  0 train_loss =  0.3227297067642212
rmse_loss =  0.5003299117088318 mae_loss =  0.41894644498825073


100%|██████████| 11/11 [00:00<00:00, 21.76it/s]


epoch =  1 train_loss =  0.3051944063468413
rmse_loss =  0.4808167815208435 mae_loss =  0.399260550737381


100%|██████████| 11/11 [00:00<00:00, 21.23it/s]


...

epoch =  47 train_loss =  0.11614006898845687
rmse_loss =  0.27950429916381836 mae_loss =  0.23615039885044098


100%|██████████| 11/11 [00:00<00:00, 21.18it/s]


epoch =  48 train_loss =  0.11479586343486588
rmse_loss =  0.27833226323127747 mae_loss =  0.23514583706855774


100%|██████████| 11/11 [00:00<00:00, 21.18it/s]

epoch =  49 train_loss =  0.11347914099693299
rmse_loss =  0.2768402099609375 mae_loss =  0.2338794618844986
